In [10]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

In [11]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = True
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta


In [12]:
######################
## D. Leer los jsons
######################

# 1. Leer los jsons y guardarlos en un dataframe
df_renacyt= pd.read_parquet(os.path.join(os.getcwd(),'outputs','parquets','renacyt.parquet'))

# 2. Generar lista de urls a partir de ['ctiVitae']
urls = []
cti_vitaes = df_renacyt['ctiVitae'].tolist()
for cit_vitae in cti_vitaes:
    url = f'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador={cit_vitae}'
    urls.append(url)

#urls = list(set(urls))
print(f'Se generaron {len(urls)} urls')

urls = urls[0:10]
urls

Se generaron 7151 urls


['https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=10',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=13',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=16',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=17',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=19',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=21',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=25',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=26',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=34',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=43']

In [13]:
######################
## E. Ejecutar
######################

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iterar sobre las urls
i = 0
for url in urls: 
    try:
        # A. Iniciar har
        start_har(proxy, 'renacyt_profile')
        
        # B. Ingresar a la url
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "elementor-widget-container")))
        print(f'{i+1}. {url}')

        # C. Obtener el har
        time.sleep(3)
        har = proxy.har

        # D. Guardar el har
        for entrie in har.get('log').get('entries'):
            if '"formacionesAcademicas"' in str(entrie):
                string_content = entrie.get('response').get('content').get('text')
                data_list = json.loads(string_content)
                with open(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile','profile_' + str(i).zfill(2) + '_data.json'), 'w') as json_file:
                    json.dump(data_list, json_file, indent=4)
                
        # E. Actualizar el contador
        i += 1
    except:
        pass

Se inicia el servidor en Mac o Linux
Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver
1. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=10
2. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=13
3. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=16
4. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=17
5. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=19
6. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=21
7. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=25
8. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=26
9. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=34
10. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=43


In [17]:
stop_chrome_driver(driver, server, proxy)

# Eliminar los archivos bmp.log y server.log
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
